In [ ]:
from dotenv import load_dotenv
import os
import pandas as pd
import altair as alt
import alpaca_trade_api as tradeapi
import requests
import json
import time
load_dotenv(dotenv_path="Api_keys.env")

In [29]:
key = os.getenv("alphavantage")
key

'UACKLUCYFFCOTJ55'

In [ ]:
   # tickers to gather data for
# separated by industry
corp_list = [ "AAL","XOM", "PFE"]


for symbol in corp_list:
    # makes request to alphavantage to get fundamental data
    content = requests.get(url= "https://www.alphavantage.co/query?",  params = {"function":"BALANCE_SHEET", "symbol": symbol, "apikey": key})
    data = content.json()
data

In [24]:
 # convert request json to dataframe
data_test = data['quarterlyReports']
df = pd.DataFrame.from_dict(data_test)
df.set_index('fiscalDateEnding', inplace=True)
file_name = f'Corporations/{symbol}_quarterly_reports.csv'
df.to_csv(file_name)
file_name


'Corporations/PFE_quarterly_reports.csv'

In [25]:
Tech_list = ["AAPL","AMZN","FB"]
for symbol in Tech_list:
    # makes request to alphavantage to get fundamental data
    content = requests.get(url= "https://www.alphavantage.co/query?",  params = {"function":"BALANCE_SHEET", "symbol": symbol, "apikey": key})
    data = content.json()
# convert request json to dataframe
    data_test = data['quarterlyReports']
    df = pd.DataFrame.from_dict(data_test)
    df.set_index('fiscalDateEnding', inplace=True)
    file_name = f'Tech/{symbol}_quarterly_reports.csv'
    df.to_csv(file_name)
    file_name

In [27]:
symbol_list = ["AAPL","AAL","AMZN", "FB","XOM","PFE"]
for symbol in symbol_list:
    stock_price = requests.get('https://www.alphavantage.co/query?', params={'function':'TIME_SERIES_MONTHLY_ADJUSTED','symbol':symbol,
                                                                              'apikey':'Alpha_vantage_Api_key'})
    stock_price = stock_price.json()
    try:
        stock_price = stock_price['Monthly Adjusted Time Series']
    except KeyError:
        print('KeyError. Trying again in 60 seconds...')  
        time.sleep(60)
        stock_price = requests.get('https://www.alphavantage.co/query?', params={'function':'TIME_SERIES_MONTHLY_ADJUSTED','symbol':symbol,
                                                                              'apikey':'Alpha_vantage_Api_key'})
        stock_price = stock_price['Monthly Adjusted Time Series']
    data = {}
    date_list = []
    close_list = []
    # Added high list and low list
    high_list = []
    low_list = []
    for date in stock_price:
        data[date] = stock_price[date]['4. close']
        date_list.append(date)
        close_list.append(stock_price[date]['4. close'])
        # Take high and low from the data pulled in
        high_list.append(stock_price[date]['2. high'])
        low_list.append(stock_price[date]['3. low'])
    # Added high and low columns into our data for cleaning and formating 
    data = {'date': date_list, 'close': close_list, 'high': high_list, 'low': low_list}
    price_df = pd.DataFrame.from_dict(data)
    price_df.columns = ['date', 'close', 'high', 'low']
    price_df.set_index('date', inplace=True)
    price_df_ascending = price_df.sort_values(by="date", ascending=True)
    price_df_average = price_df_ascending.rolling(3).mean() 
    price_df_average = price_df_ascending.iloc[::3, :].truncate(before = "2019-03-29")
    final_month = price_df_ascending.iloc[-1] # we wanna append final_month to price_df_average
    price_df_average.append(final_month) 
    file_name = f'Prices/{symbol}.csv'
    price_df.to_csv(file_name)
    print(f'{symbol}:')
    print(price_df_average)
    print('---')
    print(final_month)

AAPL:
               close      high       low
date                                    
2019-03-29  189.9500  197.6900  169.5000
2019-06-28  197.9200  201.5700  170.2700
2019-09-30  223.9700  226.4200  204.2200
2019-12-31  293.6500  293.9700  256.2900
2020-03-31  254.2900  304.0000  212.6100
2020-06-30  364.8000  372.3800  317.2100
2020-09-30  115.8100  137.9800  103.1000
2020-12-31  132.6900  138.7890  120.0100
---
close    126.6000
high     133.6116
low      126.3820
Name: 2021-01-06, dtype: object
AAL:
              close     high      low
date                                 
2019-04-30  34.1800  35.2400  31.7000
2019-07-31  30.5100  34.9900  30.2100
2019-10-31  30.0600  31.6700  24.5100
2020-01-31  26.8400  29.2950  25.3700
2020-04-30  12.0100  13.4000   9.0900
2020-07-31  11.1200  14.2900  10.8000
2020-10-30  11.2800  13.6800  10.6300
2021-01-06  15.5200  15.8800  14.8700
---
close    15.5200
high     15.8800
low      14.8700
Name: 2021-01-06, dtype: object
AMZN:
                

TypeError: 'Response' object is not subscriptable